In [1]:
import pandas as pd

In [2]:
customer_df = pd.read_csv("customer.csv")
cust_order_df = pd.read_csv("cust_order.csv")
order_history_df = pd.read_csv("order_history.csv")
order_status_df = pd.read_csv("order_status.csv")

In [3]:
cust_order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7550 entries, 0 to 7549
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            7550 non-null   int64 
 1   order_date          7550 non-null   object
 2   customer_id         7550 non-null   int64 
 3   shipping_method_id  7550 non-null   int64 
 4   dest_address_id     7550 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 295.1+ KB


In [4]:
cust_order_df["order_date"] = pd.to_datetime(cust_order_df["order_date"]).dt.date
cust_order_df["order_date"] = pd.to_datetime(cust_order_df["order_date"])

In [5]:
cust_order_df.sort_values(by=['customer_id', 'order_date'], ascending=[True, True], inplace=True)

In [6]:
cust_order_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7550 entries, 2176 to 2289
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            7550 non-null   int64         
 1   order_date          7550 non-null   datetime64[ns]
 2   customer_id         7550 non-null   int64         
 3   shipping_method_id  7550 non-null   int64         
 4   dest_address_id     7550 non-null   int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 353.9 KB


In [7]:
#find order frequency
cust_order_df['order_diff'] = cust_order_df.groupby('customer_id')['order_date'].diff().dt.days
cust_order_df.fillna(0,inplace = True)

avg_order_diff = cust_order_df.groupby('customer_id')['order_diff'].mean().reset_index()
avg_order_diff.rename(columns={'order_diff': 'Average_time_between_orders'}, inplace=True) #order_frequency

cust_order_df = cust_order_df.merge(avg_order_diff, on='customer_id', how='left')

In [8]:
# Last Order date
from datetime import datetime
last_order = cust_order_df.groupby('customer_id')['order_date'].max().reset_index()
last_order['days_since_last_order'] = (datetime.now() - last_order['order_date']).dt.days
cust_order_df = cust_order_df.merge(last_order[['customer_id', 'days_since_last_order']], on='customer_id', how='left')

In [9]:
#order count
order_count = cust_order_df.groupby('customer_id')['order_id'].count()
cust_order_df['order_count'] = cust_order_df['customer_id'].map(order_count)

In [10]:
cust_order_df.drop(["order_date","shipping_method_id","dest_address_id","order_diff"],axis=1,inplace= True)

In [11]:
cust_order_df["Average_time_between_orders"]=cust_order_df["Average_time_between_orders"].astype(int)

In [12]:
#cancelled and reteunred order
issues_df = pd.merge(cust_order_df, order_history_df, on="order_id")
issues_df = issues_df[issues_df['status_id'].isin([5, 6])]

In [13]:
cancelled_returned_counts = issues_df.groupby('customer_id')['status_id'].count().reset_index()
cancelled_returned_counts.rename(columns={'status_id': 'cancelled_returned_counts'}, inplace=True) 

In [14]:
df=pd.merge(cust_order_df,cancelled_returned_counts,on="customer_id",how="outer")

In [15]:
df.drop(["order_id"],axis=1,inplace= True)

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.fillna(0,inplace=True)

In [18]:
df["cancelled_returned_counts"]=df["cancelled_returned_counts"].astype(int)

In [19]:
df.drop(["customer_id"],axis=1,inplace= True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1682 entries, 0 to 7549
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Average_time_between_orders  1682 non-null   int64
 1   days_since_last_order        1682 non-null   int64
 2   order_count                  1682 non-null   int64
 3   cancelled_returned_counts    1682 non-null   int64
dtypes: int64(4)
memory usage: 65.7 KB


In [21]:
churn_threshold = 210  # days, adjust as needed
df['churn'] = df['days_since_last_order'].apply(lambda x: 1 if x > churn_threshold else 0)

In [22]:
df

,Average_time_between_orders,days_since_last_order,order_count,cancelled_returned_counts,churn
0,0,518,1,0,1
1,320,116,3,1,0
4,0,958,1,0,1
5,154,134,6,0,0
11,34,417,5,0,1
...,...,...,...,...,...
7526,85,93,7,2,0
7533,80,100,7,1,0
7540,296,198,3,1,0
7543,83,423,6,1,1


In [23]:
df["churn"].value_counts()
#1-churned
#0-retained

churn
0    856
1    826
Name: count, dtype: int64

In [24]:
df

,Average_time_between_orders,days_since_last_order,order_count,cancelled_returned_counts,churn
0,0,518,1,0,1
1,320,116,3,1,0
4,0,958,1,0,1
5,154,134,6,0,0
11,34,417,5,0,1
...,...,...,...,...,...
7526,85,93,7,2,0
7533,80,100,7,1,0
7540,296,198,3,1,0
7543,83,423,6,1,1


In [25]:
df.min()

Average_time_between_orders     0
days_since_last_order          11
order_count                     1
cancelled_returned_counts       0
churn                           0
dtype: int64

In [26]:
df.reset_index()

,index,Average_time_between_orders,days_since_last_order,order_count,cancelled_returned_counts,churn
0,0,0,518,1,0,1
1,1,320,116,3,1,0
2,4,0,958,1,0,1
3,5,154,134,6,0,0
4,11,34,417,5,0,1
...,...,...,...,...,...,...
1677,7526,85,93,7,2,0
1678,7533,80,100,7,1,0
1679,7540,296,198,3,1,0
1680,7543,83,423,6,1,1


In [27]:
df.to_csv("churn_prediction.csv",index=False)